In [36]:
!pip uninstall mex -y

Found existing installation: mex 0.3
Uninstalling mex-0.3:
  Successfully uninstalled mex-0.3


In [37]:
!pip install --quiet "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-2-segunda-parte-caroacostatovany.git#egg=mex&subdirectory=src"

In [1]:
#np.set_printoptions(threshold=sys.maxsize)
import os
import sys
import pytz
import timeit
import datetime
import numpy as np
import scipy.io as sio

#scipy
from scipy.optimize import linprog
from pytest import approx

#mex
from mex.simplex import simplex_networks as mex_sn
from mex.simplex import problem_definition as mex_pd
from mex.utils import general as mex_g
from mex.simplex.minimizer_class import Minimizer
from mex.simplex.maximizer_class import Maximizer

## Características de la instancia

Utilizamos una instancia m5.2xlarge que entra en el programa de AWS Educate y que hemos ocupado en prácticas anteriores. Esta instancia tiene las siguientes características:

In [39]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:            7
CPU MHz:             3100.040
BogoMIPS:            4999.96
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand h

In [2]:
mat = sio.loadmat('../data/AGG.mat')      #Start time:  09:52:51 PM  End time:  09:52:54 PM
#mat = sio.loadmat('../data/DEGEN3.mat')   #Start time:  10:27:30 PM  End time:  10:30:58 PM
#mat = sio.loadmat('../data/SHIP04L.mat')  #Start time:  09:21:59 PM  End time:  09:21:59 PM
#mat = sio.loadmat('../data/AGG2.mat')     #Start time:  09:53:47 PM  End time:  09:53:53 PM
#mat = sio.loadmat('../data/QAP8.mat')     #Start time:  10:11:35 PM  End time:  10:11:36 PM
#mat = sio.loadmat('../data/QAP15.mat')    #Start time:  10:15:30 PM  End time:  10:15:50 PM
#mat = sio.loadmat('../data/TRUSS.mat')    #Start time:  10:24:53 PM  End time:  10:24:54 PM

In [3]:
A = mat['A']
b = mat['b']
c = mat['c']
lbounds = mat['lbounds']
ubounds = mat['ubounds']
A.shape

(488, 615)

# Prueba de paquetería `mex`

In [4]:
A = A.toarray()

In [47]:
minim = Minimizer(A, b, c)

In [48]:
minim.add_constraints(lbounds,ubounds)

In [49]:
minim.matrix.shape

(1719, 2335)

In [50]:
%timeit -n 1 -r 1 minim.solve()

1min 18s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


**Valores aproximados**

In [51]:
min_approx = minim.get_min()

In [52]:
coeff_approx = minim.get_coeff()

In [54]:
min_approx

-3.602879999999999e+35

**Valores objetivo**

In [55]:
tableau_obj = minim.matrix

In [56]:
n_restr = tableau_obj.shape[0]-1
n_vars = A.shape[1]

In [57]:
c_min_obj = tableau_obj[-1,0:n_vars]

In [58]:
A_min_obj = tableau_obj[0:n_restr, 0:n_vars]

In [59]:
b_min_obj = tableau_obj[0:n_restr, -1]

In [60]:
%timeit -n 1 -r 1 min_obj = linprog(c_min_obj, A_ub=A_min_obj, b_ub=b_min_obj).fun

4min 50s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [61]:
coeff_obj = linprog(c, A_ub=A, b_ub=b).x

**Comprobación**

In [45]:
assert min_obj == approx(min_approx, rel=1e-1), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,3) == approx(coeff_approx, abs=1e+40), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -3.553890952290583e+35
El valor aproximado obtenido con mex es:  -3.602879999999999e+35


## Prueba compilación a C

In [15]:
#mex_c
os.chdir("..")
from mex.mex_c import general_c as mex_c_g
from mex.mex_c import simplex_networks_c as mex_c_sn
from mex.mex_c import problem_definition_c as mex_c_pd
#from mex.mex_c import minimizer_class_c
#from mex.mex_c import maximizer_class_c
#from mex.mex_c.minimizer_class_c import Minimizer_c
#from mex.mex_c.maximizer_class_c import Maximizer_c
os.chdir("notebooks")

In [9]:
minim_c = Minimizer_c(A,b,c)

In [ ]:
minim_c.add_constraints(lbounds,ubounds)

In [7]:
minim_c.matrix.shape

(489, 1105)

In [8]:
%timeit -n 1 -r 1 minim_c.solve()

Start time:  11:07:29 PM
End time:  11:07:32 PM


**Valores aproximados**

In [9]:
min_approx_c = minim_c.get_min()
coeff_approx_c = minim_c.get_coeff()

In [21]:
#coeff_approx_c

**Comprobación**

In [13]:
assert min_obj == approx(min_approx_c, abs=1e+5), "El valor aproximado es incorrecto"
#assert np.round(coeff_obj,5) == approx(coeff_approx_c, abs=1e+8), "El valor de los coeficientes aproximados es incorrecto"

print("El valor objetivo obtenido con scipy es: ", min_obj)
print("El valor aproximado obtenido con mex es: ", min_approx_c)
#print("Los coeficientes objetivos obtenidos con scipy son: ", coeff_obj)
#print("Los coeficientes aproximados obtenidos con mex son: ", coeff_approx)

El valor objetivo obtenido con scipy es:  -81917191.67681439
El valor aproximado obtenido con mex es:  -81917193.41974491
